In [10]:
import predictlol as pl
import config

import numpy as np

api_key = config.api_key

username = 'delphinus6'
region = 'na1'
match_index = 1


summoner_name, summoner_id, account_id = pl.get_summoner_ids(username, api_key, region)

match_data = pl.get_match_by_id(account_id, match_index, api_key, region)

num_players = len(match_data['participantIdentities']) / 2
queue_id = match_data['queueId']
queue_type = pl.queue_name(queue_id)

print('Queue type: %s' % queue_type)
print('Queue id: %d' % queue_id)
print('----')

team1 = []
team1_w = 0
team1_tot = 0
team2 = []
team2_w = 0
team2_tot = 0
for i, participant in enumerate(match_data['participantIdentities']):
    if i == 0:
        print('Team 1:')
    elif i == num_players:
        print('\nTeam 2:')

    summoner_name = participant['player']['summonerName']
    summoner_id = participant['player']['summonerId']
    
    wins, losses, tier, rank = pl.get_league_stats(summoner_id, queue_type, api_key, region)
    winrate = wins / (wins + losses)
    
    if match_data['teams'][0]['teamId'] == match_data['participants'][i]['teamId']:
#     if i < num_players:
        team1.append(winrate)
        team1_w += wins
        team1_tot += (wins + losses)
    else:
        team2.append(winrate)
        team2_w += wins
        team2_tot += (wins + losses)
    print('%s: %s/%s (%.02f%%), %s %s' % (summoner_name, wins, wins + losses, winrate * 100, tier, rank ))

print('----')

team1_sum = sum(team1)
team2_sum = sum(team2)
print('Predictions (Team 1 vs Team2):\n')
print('    Cumulative winrates: %.02f%% vs %.02f%%' % (team1_sum*100, team2_sum*100))
team1_win = match_data['teams'][0]['win'] == 'Win'
# team1_predict = team1_sum > team2_sum

sum_ratio = team1_sum / team2_sum
print('    Win likelihood from sums: %.02f%% vs %.02f%%' % (1/(1+1/sum_ratio)*100, 1/(1+sum_ratio)*100))

# Sort for min
team1.sort(reverse = False)
team2.sort(reverse = False)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
print('    Win likelihood from mean min: %.02f%% vs %.02f%%' % (1/(1+1/team1_ratio)*100, 1/(1+team1_ratio)*100))

# Sort for max
team1.sort(reverse = False)
team2.sort(reverse = True)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
max_lh_1 = 1/(1+1/team1_ratio)*100
max_lh_2 = 1/(1+team1_ratio)*100
print('    Win likelihood from mean max: %.02f%% vs %.02f%%' % (max_lh_1, max_lh_2))

map_diff = 0.026
print('    After map side difference (%.02f%%): %.02f%% vs %.02f%%' % (map_diff, max_lh_1 * (1 - map_diff), max_lh_2 * (1 + map_diff)))

team1_carry = team1[-1]/team2[-1]
team2_carry = team2[0]/team1[0]
min_carry = min(team1_carry, team2_carry)
print('    Solo carry likelihood: %.02f vs %.02f' % (team1_carry/min_carry, team2_carry/min_carry))

if team1_win:
    winner = 'Team 1'
else:
    winner = 'Team 2'
print('\nActual result: %s won!' % winner) 
# print('-->', team1_win == team1_predict)
    

    
    
    
    

Queue type: RANKED_FLEX_TT
Queue id: 470
----
Team 1:
LANCE THE WOLF: 111/232 (47.84%), SILVER II
delphinus6: 47/86 (54.65%), SILVER III
Crownless Exalt: 104/223 (46.64%), SILVER II

Team 2:
SmellMyStink: 8/16 (50.00%), SILVER II
StarbÍzno: 8/15 (53.33%), SILVER I
Starbizno: 8/16 (50.00%), SILVER I
----
Predictions (Team 1 vs Team2):

    Cumulative winrates: 149.13% vs 153.33%
    Win likelihood from sums: 49.31% vs 50.69%
    Win likelihood from mean min: 48.51% vs 51.49%
    Win likelihood from mean max: 48.51% vs 51.49%
    After map side difference (0.03%): 47.25% vs 52.82%
    Solo carry likelihood: 1.00 vs 1.05

Actual result: Team 2 won!


In [10]:
# REALTIME
import predictlol as pl
import config

import sys
import numpy as np

api_key = config.api_key

username = 'C9 Sneaky'
region = 'na1'

# Somewhat arbitrary?
map_diff = 0.026

summoner_name, summoner_id, account_id = pl.get_summoner_ids(username, api_key, region)

match_data, status_code = pl.get_current_match(summoner_id, api_key, region)
if status_code == 404:
    raise Exception('Summoner not currently in game!')
    
num_players = len(match_data['participants']) / 2
queue_id = match_data['gameQueueConfigId']
queue_type = pl.queue_name(queue_id)

print('Queue type: %s' % queue_type)
print('Queue id: %d' % queue_id)
print('----')

team1 = []
team1_w = 0
team1_tot = 0
team2 = []
team2_w = 0
team2_tot = 0
for i, participant in enumerate(match_data['participants']):
    if i == 0:
        print('Team 1:')
    elif i == num_players:
        print('\nTeam 2:')

    summoner_name = participant['summonerName']
    summoner_id = participant['summonerId']
    
    wins, losses, tier, rank = pl.get_league_stats(summoner_id, queue_type, api_key, region)
    winrate = wins / (wins + losses)
    
#     if match_data['teams'][0]['teamId'] == match_data['participants'][i]['teamId']:
    if i < num_players:
        team1.append(winrate)
        team1_w += wins
        team1_tot += (wins + losses)
    else:
        team2.append(winrate)
        team2_w += wins
        team2_tot += (wins + losses)
    print('%s: %s/%s (%.02f%%), %s %s' % (summoner_name, wins, wins + losses, winrate * 100, tier, rank ))

print('----')

team1_sum = sum(team1)
team2_sum = sum(team2)
print('Predictions (Team 1 vs Team2):\n')
print('    Cumulative winrates: %.02f%% vs %.02f%%' % (team1_sum*100, team2_sum*100))

sum_ratio = team1_sum / team2_sum
print('    Win likelihood from sums: %.02f%% vs %.02f%%' % (1/(1+1/sum_ratio)*100, 1/(1+sum_ratio)*100))

# Sort for min
team1.sort(reverse = False)
team2.sort(reverse = False)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
print('    Win likelihood from mean min: %.02f%% vs %.02f%%' % (1/(1+1/team1_ratio)*100, 1/(1+team1_ratio)*100))

# Sort for max
team1.sort(reverse = False)
team2.sort(reverse = True)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
max_lh_1 = 1/(1+1/team1_ratio)*100
max_lh_2 = 1/(1+team1_ratio)*100
print('    Win likelihood from mean max: %.02f%% vs %.02f%%' % (max_lh_1, max_lh_2))

print('    After map side difference (%.02f%%): %.02f%% vs %.02f%%' % (map_diff, max_lh_1 * (1 - map_diff), max_lh_2 * (1 + map_diff)))

team1_carry = team1[-1]/team2[-1]
team2_carry = team2[0]/team1[0]
min_carry = min(team1_carry, team2_carry)
print('    Solo carry likelihood: %.02f vs %.02f' % (team1_carry/min_carry, team2_carry/min_carry))


Queue type: RANKED_SOLO_5x5
Queue id: 420
----
Team 1:
Piggy Kitten: 901/1768 (50.96%), MASTER I
Erry: 227/425 (53.41%), MASTER I
Kumo9: 375/710 (52.82%), CHALLENGER I
Tony Mid: 385/710 (54.23%), MASTER I
JuIius: 974/1909 (51.02%), CHALLENGER I

Team 2:
C9 Sneaky: 498/956 (52.09%), MASTER I
Tragedy 3: 420/813 (51.66%), MASTER I
OFFLINETV REJECT: 267/501 (53.29%), MASTER I
OpTic Arrow: 404/758 (53.30%), CHALLENGER I
Julien: 575/1099 (52.32%), CHALLENGER I
----
Predictions (Team 1 vs Team2):

    Cumulative winrates: 262.44% vs 262.66%
    Win likelihood from sums: 49.98% vs 50.02%
    Win likelihood from mean min: 49.95% vs 50.05%
    Win likelihood from mean max: 49.95% vs 50.05%
    After map side difference (0.03%): 48.65% vs 51.36%
    Solo carry likelihood: 1.00 vs 1.00


In [24]:
import requests
summoner_name = 'delphinus6'
response = requests.get("https://%s.api.riotgames.com/lol/spectator/v3/active-games/by-summoner/%s?api_key=%s" % (region, summoner_id, api_key))

In [25]:
response.status_code

200

In [27]:
response.headers

{'Accept-Ranges': 'bytes', 'Age': '175', 'Cache-Control': 'no-cache, max-age=180', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json;charset=utf-8', 'Date': 'Fri, 20 Jul 2018 07:40:45 GMT', 'Expires': 'Wed, 31 Dec 1969 16:00:00 PST', 'Pragma': 'No-cache', 'Via': '1.1 varnish', 'X-App-Rate-Limit': '20:1,100:120', 'X-App-Rate-Limit-Count': '1:1,5:120', 'X-Method-Rate-Limit': '20000:10,1200000:600', 'X-Method-Rate-Limit-Count': '1:10,1:600', 'X-Varnish': '1888135362 1888133252', 'transfer-encoding': 'chunked', 'Connection': 'keep-alive'}